In [9]:
import sys
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf


In [10]:
# Plot settings is a custom module which sets the plot style
sys.path.append(os.path.abspath("../src/visualization"))
from plot_settings import set_plot_style

set_plot_style()


## 1.1 Read 

In [5]:
X_train, y_train, X_val, y_val, X_test = pd.read_pickle("../data/processed/data.pkl")
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape


((37800, 28, 28, 1),
 (37800, 10),
 (4200, 28, 28, 1),
 (4200, 10),
 (28000, 28, 28, 1))

#####  our model was included custom function(f1_m, precision_m, recall_m). when we want use "load_model" we need to mention that custom_object and import them to this file

In [8]:
sys.path.append(os.path.abspath("../src/models"))
from metrics import f1_m, precision_m, recall_m

model = tf.keras.models.load_model(
    "../models/final_model.h5",
    custom_objects={"f1_m": f1_m, "precision_m": precision_m, "recall_m": recall_m},
)


## 2.1 Validation results